In [1]:
import re
import json
import csv
from io import StringIO
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
url_esg = 'https://finance.yahoo.com/quote/{}/sustainability?p={}'

In [3]:
data = pd.read_csv(r"C:\Users\18143\OneDrive\Desktop\tickers_6123.csv")   
df = pd.DataFrame(data)
df=df.dropna()

In [4]:
Ticker = list(df['Ticker'])

In [5]:
stock=Ticker[:-1]

In [6]:
esg_score=[]

In [7]:
def get_esg(stock):
    response = requests.get(url_esg.format(stock,stock))
    soup = BeautifulSoup(response.text,'html.parser')
    pattern = re.compile(r'\s--\sData\s--\s')
    try:
        script_data = soup.find('script',text=pattern).contents[0]
    except:
        return None
    start = script_data.find("context")-2
    json_data = json.loads(script_data[start:-12])
    #skipped on displaying keys
    #json_data['context'].keys()
    #json_data['context']['dispatcher']['stores']['QuoteSummaryStore'].keys()
    try:
        esgScores = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['esgScores']
    except KeyError:
        return None
    
    stat = ['peerSocialPerformance','socialScore','peerGovernancePerformance','governanceScore','peerEnvironmentPerformance','environmentScore','peerEsgScorePerformance','totalEsg','percentile']
    dic = {}

    for i in stat:
        try:
            dic[i]=esgScores[i]
        except KeyError:
            continue
    
    score = []
    peer = []
    for s in dic:
        try:
            score.append((s,dic[s]['raw']))
        except KeyError:
            continue
    for s in dic:
        try: 
            peer.append((s+'_min',dic[s]['min']))
            peer.append((s+'_avg',dic[s]['avg']))
            peer.append((s+'_max',dic[s]['max']))
        except KeyError:
            continue
    
    score=dict(score)
    peer=dict(peer)
    
    new_dic = {**score,**peer}
    new_dic['ticker']=stock
    return new_dic
    

In [8]:
get_esg('AAPL')

{'socialScore': 7.69,
 'governanceScore': 8.94,
 'environmentScore': 0.1,
 'totalEsg': 16.73,
 'percentile': 11.77,
 'peerSocialPerformance_min': 2.39,
 'peerSocialPerformance_avg': 6.449090909090907,
 'peerSocialPerformance_max': 12.2,
 'peerGovernancePerformance_min': 3.72,
 'peerGovernancePerformance_avg': 7.539090909090912,
 'peerGovernancePerformance_max': 14.1,
 'peerEnvironmentPerformance_min': 0.04,
 'peerEnvironmentPerformance_avg': 3.2439999999999998,
 'peerEnvironmentPerformance_max': 10.02,
 'peerEsgScorePerformance_min': 7.24,
 'peerEsgScorePerformance_avg': 17.05464285714286,
 'peerEsgScorePerformance_max': 26.59,
 'ticker': 'AAPL'}

In [9]:
for i in stock:
    esg_score.append(get_esg(i))

In [10]:
#removing None
res = [i for i in esg_score if i] 

In [11]:
len(res)

864

In [12]:
na_list = []
new_list=[]
for i in res:
    if len(i) != 18:
        na_list.append(i)
    else:
        new_list.append(i)

In [13]:
len(new_list)

527

In [14]:
#remove the duplicates
updated = [] 
[updated.append(x) for x in new_list if x not in updated] 
len(updated)

510

In [15]:
toCSV = new_list
keys = toCSV[0].keys()
with open('esg_score_updated.csv', 'w', newline='')  as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(toCSV)

In [18]:
toCSV = res
keys = toCSV[0].keys()
with open('esg_score_test.csv', 'w', newline='')  as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(toCSV)